In [1]:
# Adding system path
import sys, pathlib
sys.path.append(str(pathlib.Path.cwd().parent.parent))
# sys.path

In [2]:
# Set to show warnings only once
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Setting up displays
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from dash import Dash,html,dcc,Input,Output
app = Dash(__name__)
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from tabulate import tabulate
color_scheme=["red","blue","green","orange","purple","brown","pink","gray","olive","cyan","darkviolet","goldenrod","darkgreen","chocolate","lawngreen"]

In [4]:
from dash import Dash,html,dcc

In [ ]:
# ##center allign all the figure outputs.
# from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
# from IPython.display import display, HTML
# from plotly.graph_objs import *
# import numpy as np
# init_notebook_mode(connected=True)

# display(HTML("""
# <style>
# .output {
#     display: flex;
#     align-items: center;
#     text-align: center;
# }
# </style>
# """))

In [5]:
# Setting up autoreload for libs
%load_ext autoreload
%autoreload 2
%aimport optiml.queries

In [6]:
# Initialize connection to Snowflake and set analysis date
from optiml.connection import SnowflakeConnConfig
connection = SnowflakeConnConfig(accountname='jg84276.us-central1.gcp',warehousename="XSMALL_WH").create_connection()
# Initialize local environment
import os
cache_dir = os.path.expanduser('~/data/kiva')
# Initialize query library
from optiml.queries import SNFLKQuery
qlib = SNFLKQuery(connection, 'KIV', cache_dir)
sdate = '2022-10-01'
edate = '2022-10-31'
print(f"The analysis is carried our for date range {sdate} to {edate}")

Connecting...
The analysis is carried our for date range 2022-10-01 to 2022-10-31


# User Analysis

In [7]:
## This would be my starting point for users that are a security risk
idle_user_start_date = '2022-04-01'
idle_user_end_date = '2022-10-31'
df = qlib.idle_users(idle_user_start_date, idle_user_end_date)
df.head()

,name,created_on,deleted_on,login_name,email,must_change_password,disabled,snowflake_lock,default_role,last_success_login,locked_until_time,password_last_set_time
0,SHERSKOVITZ_SFC,2016-06-10 18:44:13.106000-07:00,NaT,SHERSKOVITZ_SFC,support@snowflakecomputing.com,False,"""true""","""false""",SYSADMIN,2016-07-15 00:43:15.994000-07:00,NaT,2016-07-15 00:43:24.642000-07:00
1,AMEYENDORFF_SFC,2016-06-10 18:42:29.683000-07:00,NaT,AMEYENDORFF_SFC,<NA>,False,"""true""","""false""",SYSADMIN,2017-03-29 00:49:59.647000-07:00,NaT,2016-06-10 21:03:55.279000-07:00
2,VERTEX,2016-07-11 21:39:38.257000-07:00,NaT,VERTEX,<NA>,False,"""false""","""false""",SYSADMIN,2017-06-05 23:30:03.758000-07:00,1970-01-01 00:00:00-08:00,2016-07-11 21:39:38.612000-07:00
3,NIKKI,2016-06-14 19:18:14.235000-07:00,NaT,NIKKI,nikkiw@kiva.org,False,"""false""","""false""",SYSADMIN,2017-07-31 23:54:26.579000-07:00,NaT,2016-06-14 19:20:05.593000-07:00
4,NAGIOS_DEV,2017-03-10 22:33:59.208000-08:00,NaT,NAGIOS_DEV,<NA>,False,"""false""","""false""",NAGIOS_DEV_ROLE,2017-08-29 22:14:09.729000-07:00,NaT,2017-03-13 20:55:08.113000-07:00


In [13]:
# import plotly.express as px
# data_canada = px.data.gapminder().query("country == 'Canada'")
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=df['last_success_login'], y=[0]*len(df['last_success_login'].to_list()), mode='markers', marker_size=20
))
fig.update_xaxes(showgrid=False, title_text="last sucess login")
fig.update_yaxes(showgrid=False, 
                 zeroline=True, zerolinecolor='black', zerolinewidth=3,
                 showticklabels=False)
fig.update_layout(height=200, plot_bgcolor='white', title_text="Idle users")
fig.show()

In [15]:
## Users never logged in but not deleted
idle_user_start_date = '2022-04-01'
idle_user_end_date = '2022-10-31'
df = qlib.users_never_logged_in(idle_user_start_date, idle_user_end_date)
df.head()

,name,created_on,deleted_on,login_name,email,must_change_password,disabled,snowflake_lock,default_role,last_success_login,locked_until_time,password_last_set_time
0,SPENCERM_DEV,2022-08-08 16:48:23.119000-07:00,NaT,SPENCERM_DEV,<NA>,False,"""false""","""false""",<NA>,NaT,NaT,NaT
1,YARIELI_DEV,2022-01-10 20:08:17.338000-08:00,NaT,YARIELI_DEV,<NA>,True,"""false""","""false""",YARIELI_DEV_ROLE,NaT,NaT,2022-01-10 20:08:17.360000-08:00
2,TEST_EXERCISE_DV,2022-10-10 21:35:16.747000-07:00,NaT,TEST_EXERCISE_DV,<NA>,False,"""false""","""false""",TEST_EXERCISE_ROLE,NaT,NaT,2022-10-10 21:35:16.747000-07:00
3,SNOWFLAKE,2016-06-10 18:42:29.798000-07:00,NaT,SNOWFLAKE,<NA>,False,"""false""","""false""",<NA>,NaT,NaT,NaT


In [16]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=df['created_on'], y=[0]*len(df['created_on'].to_list()), mode='markers', marker_size=20
))
fig.update_xaxes(showgrid=False, title_text="Created on")
fig.update_yaxes(showgrid=False, 
                 zeroline=True, zerolinecolor='black', zerolinewidth=3,
                 showticklabels=False)
fig.update_layout(height=200, plot_bgcolor='white', title_text="Never logged in")
fig.show()

In [ ]:
df=qlib.users_full_table_scans(start_date='2018-04-01', end_date='2022-10-31')
df.head()

In [ ]:
df=qlib.heavy_users(start_date="2019-01-01", end_date="2022-02-02")
df.head()

In [17]:
df=qlib.idle_roles(start_date="2021-01-01", end_date="2022-02-02")
df.head()

,created_on,deleted_on,name,comment,owner
0,2019-10-16 04:04:13.090000-07:00,NaT,BRIANK_DEV_ROLE,<NA>,SECURITYADMIN
1,2020-09-30 21:25:57.787000-07:00,NaT,WENZ_DEV_ROLE,<NA>,SECURITYADMIN
2,2017-05-23 20:29:35.657000-07:00,NaT,VERTEX_API_QA_ROLE,<NA>,SECURITYADMIN
3,2018-03-09 20:16:09.347000-08:00,NaT,LEDLIE_DEV_ROLE,<NA>,SECURITYADMIN
4,2018-03-13 17:13:24.128000-07:00,NaT,LEOP_DEV_ROLE,<NA>,SECURITYADMIN


In [18]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=df['created_on'], y=[0]*len(df['created_on'].to_list()), mode='markers', marker_size=20
))
fig.update_xaxes(showgrid=False, title_text="Created on")
fig.update_yaxes(showgrid=False, 
                 zeroline=True, zerolinecolor='black', zerolinewidth=3,
                 showticklabels=False)
fig.update_layout(height=200, plot_bgcolor='white', title_text="Idle roles")
fig.show()

In [ ]:
## Warehouse credit usage over time



app = Dash(__name__)



fig = px.scatter(df_expensiveq, x=df_expensiveq["query_id"], y=df_expensiveq["relative_performance_cost"],
                 color=df_expensiveq["user_name"], hover_name=df_expensiveq["warehouse_size"],size_max=30)
fig.update_traces(marker_size=20)


app.layout = html.Div([
    dcc.Graph(
        id='life-exp-vs-gdp',
        figure=fig
    )
])

fig.show()

In [ ]:
app = Dash(__name__)



app.layout = html.Div([
    html.Div([

        html.Div([
            dcc.Dropdown(
                df['user_name'].unique(),
                'User_name',
                id='xaxis-column'
            ),
            dcc.RadioItems(
                ['Linear', 'Log'],
                'Linear',
                id='xaxis-type',
                inline=True
            )
        ], style={'width': '48%', 'display': 'inline-block'}),

        html.Div([
            dcc.Dropdown(
                df['Indicator Name'].unique(),
                'Life expectancy at birth, total (years)',
                id='yaxis-column'
            ),
            dcc.RadioItems(
                ['Linear', 'Log'],
                'Linear',
                id='yaxis-type',
                inline=True
            )
        ], style={'width': '48%', 'float': 'right', 'display': 'inline-block'})
    ]),

    dcc.Graph(id='indicator-graphic'),

    dcc.Slider(
        df['Year'].min(),
        df['Year'].max(),
        step=None,
        id='year--slider',
        value=df['Year'].max(),
        marks={str(year): str(year) for year in df['Year'].unique()},

    )
])


@app.callback(
    Output('indicator-graphic', 'figure'),
    Input('xaxis-column', 'value'),
    Input('yaxis-column', 'value'),
    Input('xaxis-type', 'value'),
    Input('yaxis-type', 'value'),
    Input('year--slider', 'value'))
def update_graph(xaxis_column_name, yaxis_column_name,
                 xaxis_type, yaxis_type,
                 year_value):
    dff = df[df['Year'] == year_value]

    fig = px.scatter(x=dff[dff['Indicator Name'] == xaxis_column_name]['Value'],
                     y=dff[dff['Indicator Name'] == yaxis_column_name]['Value'],
                     hover_name=dff[dff['Indicator Name'] == yaxis_column_name]['Country Name'])

    fig.update_layout(margin={'l': 40, 'b': 40, 't': 10, 'r': 0}, hovermode='closest')

    fig.update_xaxes(title=xaxis_column_name,
                     type='linear' if xaxis_type == 'Linear' else 'log')

    fig.update_yaxes(title=yaxis_column_name,
                     type='linear' if yaxis_type == 'Linear' else 'log')

    return fig